# Execution tutorial

This tutorial covers the basics of executing a quantum circuit using Classiq directly through the Python SDK. It is also possible to use the [Classiq Platform](https://platform.classiq.io) to execute quantum algorithms.

For this, we will start by synthesizing the following example, from the [synthesis tutorial](https://docs.classiq.io/latest/explore/tutorials/basic_tutorials/the_classiq_tutorial/synthesis_tutorial/):

In [ ]:
from classiq import (
    Output,
    Preferences,
    QNum,
    allocate,
    hadamard_transform,
    qfunc,
    show,
    synthesize,
)
from classiq.execution import (
    ExecutionJob,
    ExecutionSession,
    set_quantum_program_execution_preferences,
)


@qfunc
def main(x: Output[QNum[3]], y: Output[QNum]) -> None:
    allocate(x)
    hadamard_transform(x)
    y |= x**2 + 1


qprog = synthesize(main)
show(qprog)

Quantum program link: https://platform.classiq.io/circuit/2xjOP268lmVtmDjQa5sTlNiFMK8?login=True&version=0.81.0


This quantum program evaluates the function $y(x) = x^2 + 1$, for all integers $x \in [0,7]$. To execute a quantum program and save its results in the Python SDK, create an `ExecutionSession`. To sample the states using this object, one can use `sample`:

In [5]:
with ExecutionSession(qprog) as es:
    results = es.sample()

The outputs of the quantum circuit can be obtained through the attributes `counts` and `parsed_counts`.

In [7]:
print("Output of the executing using counts: ", results.counts)

print(
    "Output of the executing using parsed_counts: ",
    results.parsed_counts,
)

Output of the executing using counts:  {'110010111': 247, '100101110': 263, '010001100': 238, '001010011': 262, '000010001': 254, '000101010': 247, '000001000': 269, '011010101': 268}
Output of the executing using parsed_counts:  [{'x': 0, 'y': 1}: 269, {'x': 5, 'y': 26}: 268, {'x': 6, 'y': 37}: 263, {'x': 3, 'y': 10}: 262, {'x': 1, 'y': 2}: 254, {'x': 7, 'y': 50}: 247, {'x': 2, 'y': 5}: 247, {'x': 4, 'y': 17}: 238]


As we can see, the results from the execution are the same but differ in format: 

* `counts` will output a `dict` containing the bitstrings measured followed by its number of counts.
* `parsed_counts` will output a `list` of `dicts`, each containing the values of `x` and `y` followed by its number of counts.

These two ways of reading the results can be used in different.

## Backend selection

The backend of an execution is the hardware choice where the quantum program is executed. It can be classical hardware, i.e., a simulator, or real hardware. To select a specific backend, it is necessary to import its correct Backend Preferences from `classiq.execution`. Check the different [Cloud Providers](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) and their Backend Preferences for execution.

In this section we will explore two different examples for fixation: 

### First example: Execution using the state vector simulator from Classiq

Since Classiq provides its own state vector simulator backend, we will use `ClassiqBackendPreferences` to define it as the state vector simulator. This information is provided on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/).

To define the quantum program's execution preferences, use `ExecutionPreferences` and then `set_quantum_program_execution_preferences`. In this example, we will perform a simulation with `num_shots=1` since the state vector simulator performs an exact simulation of the quantum program.

If no backend is defined on the preferences, then the [Classiq simulator](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/classiq-backends/) is selected.


In [ ]:
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences

backend_preferences = ClassiqBackendPreferences(
    backend_name="simulator_statevector"
)  # Always check the Cloud Providers to correctly define the backend.

execution_preferences = ExecutionPreferences(
    num_shots=1, backend_preferences=backend_preferences
)

set_quantum_program_execution_preferences(qprog, execution_preferences)

QuantumProgram(outputs={<QuantumFormat.QASM: 'qasm'>: "// G...[8];" (length=4181)}, qasm_version=<QasmVersion.V2: '2.0'>, version='0.80.1', interface_version='11', hardware_data=SynthesisHardwareData(basis_gates=['id', 'r', 'u1', 'rz', 'tdg', 'cx', 's', 'y', 'u', 'u2', 'x', 'p', 'cz', 't', 'ry', 'h', 'sx', 'sdg', 'rx', 'z', 'sxdg', 'cy'], connectivity_map=None, is_symmetric_connectivity=True, backend_data=None), initial_values=None, data=GeneratedCircuitData(width=9, circuit_parameters=[], qubit_mapping=QubitMapping(logical_inputs={}, logical_outputs={'x': (0, 1, 2), 'y': (3, 4, 5, 6, 7, 8)}, physical_inputs={}, physical_outputs={'x': (0, 1, 2), 'y': (3, 4, 5, 6, 7, 8)}), execution_data=None), model=ExecutionModel(version='0.80.1', interface_version='11', enums=[], types=[], qstructs=[], constants=[], classical_execution_code='dummy', execution_preferences=ExecutionPreferences(noise_properties=None, random_seed=4018210209, backend_preferences=ClassiqBackendPreferences(backend_service_p

Now, execute the quantum program using `execute`.

In [9]:
with ExecutionSession(qprog) as es:
    results_statevector = es.sample()

The outputs of the quantum program can be obtained through the attributes `state_vector` and `parsed_state_vector` from `results.result_value()`. Below, we can see the amplitudes for the state `'x':0, 'y':1`, which can be represented by the bitstring '0000000000001000':

In [ ]:
print(
    "Amplitude of the state 000001000: ",
    results_statevector.state_vector.get("000001000", None),
)

state = next(
    (s for s in results_statevector.parsed_state_vector if s.bitstring == "000001000"),
    None,
)
print("Amplitude of the state {'x':0, 'y':1}: ", state.amplitude if state else None)

Amplitude of the state 000001000:  (-2.5162583618884327e-15+0.3535533905932734j)
Amplitude of the state {'x':0, 'y':1}:  (-2.5162583618884327e-15+0.3535533905932734j)


The outputs from the execution obtained via statevector simulator will differ from the default simulator:

* `state_vector` will output a `dict` containing the bitstrings followed by its numerically evaluated amplitudes.
* `parsed_state_vector` will output a `list` of `SimulatedState`, each containing the values of `x` and `y` followed by its bitstrings and its numerically evaluated amplitudes.

### Second example: Execution on free access IBM Hardware

We will now execute our quantum program on `ibm_brisbane`, a quantum computer from IBM Quantum. For this, we need to check on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) for the [IBM Quantum Backends](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/ibm-backends/). First, we need to do [hardware-aware synthesis](), i.e., synthesize the quantum program accordingly to the hardware that it will be executed. This can be done by specifying the backend using the `Preferences` function.

In [ ]:
preferences = Preferences(
    backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"
)
# Uncomment the following line to set the execution preferences and modify the backend.
# qprog = synthesize(main, preferences=preferences)

On the Cloud Providers page, we can see that an access token from an IBM Quantum account is required. Using this information, it is possible to set our execution preferences:

In [ ]:
from classiq.execution import (
    ExecutionPreferences,
    IBMBackendPreferences,
    IBMBackendProvider,
)

ibm_provider = IBMBackendProvider(hub="ibm-q", group="open", Project="main")
ibm_backend_preferences = IBMBackendPreferences(
    backend_name="ibm_brisbane",
    access_token="your-access-token-here",
    provider=ibm_provider,
)

execution_preferences = ExecutionPreferences(
    num_shots=1000, backend_preferences=ibm_backend_preferences
)
# Uncomment the following line to set the execution preferences and modify the backend.
# qprog = set_quantum_program_execution_preferences(qprog, execution_preferences)

In [ ]:
with ExecutionSession(qprog) as es:
    results = es.sample()

Its outputs will formatted in the same way of the outputs of the Classiq simulator:

In [ ]:
print(
    "Output of the execution using counts: ",
    results.counts,
)

print(
    "Partial output of the execution using parsed_counts: ",
    results.parsed_counts[:10],
)

Output of the executing using counts:  {'010110100': 2, '101101101': 4, '100011110': 4, '011100011': 5, '011011110': 4, '111010001': 3, '111001010': 2, '001011110': 1, '001011010': 4, '010111011': 2, '111110111': 2, '110011100': 1, '110101101': 2, '001000011': 1, '101011110': 1, '100010000': 2, '010011111': 3, '100101111': 2, '100111100': 4, '011100001': 6, '111001011': 1, '010010000': 4, '010010111': 4, '100001010': 2, '100101000': 6, '111100010': 2, '000011001': 3, '111000101': 3, '011100100': 3, '001100001': 4, '110101011': 2, '001010001': 3, '110110100': 2, '110001010': 2, '000000110': 3, '100101100': 3, '110001110': 1, '010001101': 4, '001000110': 2, '001000100': 2, '110110000': 2, '010011010': 2, '011001101': 4, '101001001': 2, '110111001': 1, '101110101': 2, '110110001': 3, '000001000': 2, '111000111': 3, '010010011': 3, '000110000': 2, '101100101': 2, '001001101': 4, '011111110': 3, '001100000': 3, '101111100': 4, '001100100': 4, '000001001': 2, '100001100': 5, '001111001': 2, 

When executing on hardware, it is often necessary to wait while your job is pending, i.e., wait for the other jobs scheduled to be executed on that machine to finish, before running yours. For this case, it is important to submit an execution job so its possible to retrieve it later. For this end, instead of using `sample`, one can use `submit_sample`:

In [21]:
with ExecutionSession(qprog) as es:
    results = es.submit_sample()
    job_ID = results.id

Once you have a job ID, it is possible to retrieve its execution data using `ExecutionJob`:

In [ ]:
# Retrieving its information:
job_execution = ExecutionJob.from_id(job_ID)

print(
    "Output of the retrieved job using counts: ",
    job_execution.result_value().counts,
)

print(
    "Partial output of the retrieved job using parsed_counts: ",
    job_execution.result_value().parsed_counts[:10],
)

Output of the retrieved job using counts:  {'111101100': 3, '001010010': 3, '111001101': 1, '011100111': 3, '101100100': 4, '110011001': 2, '101000100': 6, '001010001': 7, '001101110': 5, '010000011': 2, '110010110': 1, '000001101': 3, '100100110': 4, '100010010': 1, '110001000': 3, '001110001': 5, '110111010': 2, '100111110': 3, '010110001': 3, '110101101': 1, '001010100': 4, '010100101': 3, '111011100': 3, '100111111': 4, '100110010': 4, '001111001': 4, '111011000': 2, '001110101': 3, '100110100': 4, '011101001': 1, '110110010': 4, '100011110': 2, '010111101': 4, '001010000': 3, '111000111': 1, '011011010': 4, '010001011': 2, '111011111': 3, '101100110': 2, '010000010': 2, '000010111': 2, '110011100': 2, '001101001': 4, '111010101': 1, '111110111': 2, '110110111': 1, '101100001': 2, '011001110': 1, '011111001': 2, '010001001': 6, '011100001': 4, '011110100': 4, '001001011': 2, '011010101': 3, '000000111': 3, '001111000': 4, '011111100': 4, '100111001': 4, '100100000': 2, '101110010':